In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import datetime
from dataclasses import make_dataclass
from typing import Any, Dict, List, Optional, Union

import pandas as pd
from loguru import logger

import fitdecode

pd.set_option("display.max_columns", 101)

In [7]:
FitReferenceField = make_dataclass(
    "FitReferenceField", fitdecode.types.ReferenceField.__slots__
)
FitFieldDefinition = make_dataclass(
    "FitFieldDefinition", fitdecode.types.FieldDefinition.__slots__
)
FitBaseType = make_dataclass("FitBaseType", fitdecode.types.BaseType.__slots__)
FitFieldType = make_dataclass("FitFieldType", fitdecode.types.FieldType.__slots__)
FitField = make_dataclass("FitField", fitdecode.types.Field.__slots__)
FitMessageType = make_dataclass("FitMessageType", fitdecode.types.MessageType.__slots__)
FitSubField = make_dataclass("FitSubField", fitdecode.types.SubField.__slots__)
FitFieldData = make_dataclass("FitFieldData", fitdecode.types.FieldData.__slots__)

FitComponentField = make_dataclass(
    "FitComponentField", fitdecode.types.ComponentField.__slots__
)

FitHeaderRow = make_dataclass("FitHeaderRow", fitdecode.records.FitHeader.__slots__)

FitDefinitionRow = make_dataclass(
    "FitDefinitionRow", fitdecode.records.FitDefinitionMessage.__slots__
)

FitDataMessageRecord = make_dataclass(
    "FitDataMessageRecord", fitdecode.records.FitDataMessage.__slots__
)


FieldTypePyType = Union[fitdecode.types.FieldType, fitdecode.types.BaseType]


def parse_referene_field(record: fitdecode.types.ReferenceField) -> FitReferenceField:
    return FitReferenceField(
        record.name, record.def_num, record.value, record.raw_value
    )


def parse_compnent_field(record: fitdecode.types.ComponentField) -> FitComponentField:
    return FitComponentField(
        record.name,
        record.def_num,
        record.scale,
        record.offset,
        record.units,
        record.accumulate,
        record.bits,
        record.bit_offset,
    )


def parse_subfield(record: fitdecode.types.SubField) -> FitSubField:
    return FitSubField(
        record.name,
        record.def_num,
        parse_field_type(record.type),
        record.scale,
        record.offset,
        record.units,
        [parse_compnent_field(field) for field in record.components]
        if record.components is not None
        else None,
        [parse_referene_field(referene_field) for referene_field in record.ref_fields]
        if record.ref_fields is not None
        else None,
    )


def parse_field_definition(
    record: fitdecode.types.FieldDefinition,
) -> FitFieldDefinition:
    return FitFieldDefinition(
        parse_field(record.field),
        record.def_num,
        parse_field_type(record.base_type),
        record.size,
    )


def parse_base_type(record: fitdecode.types.BaseType) -> FitBaseType:
    return FitBaseType(
        record.name, record.identifier, record.fmt, record.size, "record.parse"
    )


def parse_field_type(record: FieldTypePyType) -> FieldTypePyType:
    if isinstance(record, fitdecode.types.FieldType):
        return FitFieldType(
            record.name, parse_base_type(record.base_type), "record.enum"
        )
    if isinstance(record, fitdecode.types.BaseType):
        return parse_base_type(record)


def parse_field(record: fitdecode.types.Field) -> FitField:

    return FitField(
        record.name,
        parse_field_type(record.type),
        record.def_num,
        record.scale,
        record.offset,
        record.units,
        [parse_compnent_field(field) for field in record.components]
        if record.components is not None
        else None,
        [parse_subfield(subfield) for subfield in record.subfields]
        if record.subfields is not None
        else None,
    )


def parse_message_type(record: fitdecode.types.MessageType) -> FitMessageType:
    fields = {}
    for field in record.fields:
        fields[field] = parse_field(record.fields[field])
    return FitMessageType(record.name, record.mesg_num, fields)


def parse_field_data(record: fitdecode.types.FieldData) -> FitFieldData:
    return FitFieldData(
        "parse_field_definition(record.field_def)",
        parse_field(record.field),
        record.parent_field,
        record.value,
        record.raw_value,
        record.units,
    )


def parse_header_record(record: fitdecode.records.FitHeader) -> FitHeaderRow:
    return FitHeaderRow(
        record.header_size,
        record.proto_ver,
        record.profile_ver,
        record.body_size,
        record.crc,
        record.crc_matched,
        record.chunk,
    )


def parse_definition_record(
    record: fitdecode.records.FitDefinitionMessage,
) -> FitHeaderRow:
    return FitDefinitionRow(
        record.is_developer_data,
        record.local_mesg_num,
        record.time_offset,
        parse_message_type(record.mesg_type),
        record.global_mesg_num,
        record.endian,
        [parse_field_definition(field_def) for field_def in record.field_defs]
        if record.field_defs is not None
        else None,
        record.dev_field_defs,
        record.chunk,
    )


def parse_data_message_record(
    record: fitdecode.records.FitDataMessage,
) -> FitDataMessageRecord:
    return FitDataMessageRecord(
        record.is_developer_data,
        record.local_mesg_num,
        record.time_offset,
        "parse_definition_record(record.def_mesg)",
        [parse_field_data(field) for field in record.fields]
        if record.fields is not None
        else None,
        record.chunk,
    )

In [8]:
# if field.field.subfields is not None:
#     for subfield in field.field.subfields:
#         fields.append(
#             (
#                 f"{field.field.name}_subfield_{subfield.name}",
#                 get_py_type(field.field.type.name),
#             )
#         )

In [50]:
def get_py_type(field_def: fitdecode.types.FieldDefinition) -> Any:
    field: fitdecode.types.FieldType = field_def.field
    print(field)
    if "date_time" in field.name:
        return datetime.datetime
    if "int" in field.name:
        if field.scale is None:
            return int
        else:
            return float
    else:
        return str


def parse_fit_file(path: str) -> Any:
    with fitdecode.FitReader(path) as fit:
        fit_frames = [frame for frame in fit]
    fit_classes: Optional[Dict] = None
    for frame in fit_frames:
        field_defs = []
        if isinstance(frame, fitdecode.FitDefinitionMessage):
            if fit_classes == None:
                fit_classes = {}
            print(parse_definition_record(frame))
            def_name = frame.mesg_type.name
            for field_def in frame.field_defs:
                field_defs.append((field_def.field.name, get_py_type(field_def)))
                if field_def.field.components is not None:
                    for component in field_def.field.components:
                        field_defs.append(
                            (f"{component.name}", get_py_type(field_def),)
                        )
            schema = tuple(field_defs)
            if fit_classes.get(def_name, None) is None:
                fit_classes[def_name] = {"schema": schema, "data": []}
                print(f"Creating {def_name} def")
                print(f"{schema}")
            elif fit_classes[def_name]["schema"] != schema:
                record_idx = 1
                for key in fit_classes.keys():
                    if (def_name in key) and fit_classes[key][schema] != schema:
                        record_idx += 1
                    else:
                        def_name = f"{key}_{record_idx}"
                        break
                fit_classes[def_name] = {"schema": schema, "data": []}
        # read rows
        if isinstance(frame, fitdecode.FitDataMessage):
            data_row = [field.value for field in frame.fields]
            print(
                [
                    (key, datum)
                    for key, datum in zip([key for key, typ in schema], data_row)
                ]
            )
            fit_classes[def_name]["data"].append(data_row)
    return fit_classes

In [51]:
# goat = parse_fit_file("fit_files/The_Sufferfest_G_O_A_T.fit")

In [55]:
# ride = parse_fit_file("fit_files/Afternoon_Ride.fit")

In [22]:
pd.DataFrame(ride["file_id"]["data"], columns=dict(ride["file_id"]["schema"]).keys())

,type,manufacturer,product,serial_number,time_created
0,activity,the_sufferfest,1231,12345,2020-05-22 16:16:16+00:00


In [23]:
pd.DataFrame(ride["file_id"]["data"], columns=dict(ride["file_id"]["schema"]).keys())

,type,manufacturer,product,serial_number,time_created
0,activity,the_sufferfest,1231,12345,2020-05-22 16:16:16+00:00


In [24]:
pd.DataFrame(
    ride["device_info"]["data"], columns=dict(ride["device_info"]["schema"]).keys()
)

,timestamp,battery_status
0,2020-05-22 16:58:50+00:00,good


In [25]:
pd.DataFrame(ride["record"]["data"], columns=dict(ride["record"]["schema"]).keys())

,timestamp,power,speed,enhanced_speed,distance,cadence,heart_rate
0,2020-05-22 16:16:16+00:00,0,0.000,0.000,0.0,0,0
1,2020-05-22 16:16:16+00:00,0,0.000,0.000,0.0,0,0
2,2020-05-22 16:16:16+00:00,0,0.000,0.000,0.0,2,133
3,2020-05-22 16:16:17+00:00,0,0.000,0.000,0.0,2,133
4,2020-05-22 16:16:18+00:00,22,0.000,0.000,0.0,2,134
...,...,...,...,...,...,...,...
2530,2020-05-22 16:58:22+00:00,110,6.706,6.706,19808.0,92,136
2531,2020-05-22 16:58:23+00:00,111,6.706,6.706,19815.0,92,135
2532,2020-05-22 16:58:24+00:00,111,6.706,6.706,19821.0,96,135
2533,2020-05-22 16:58:25+00:00,110,6.706,6.706,19828.0,96,136


In [24]:
pd.DataFrame(goat["lap"]["data"], columns=dict(goat["lap"]["schema"]).keys())

,timestamp,start_time,total_elapsed_time,total_distance,max_speed,enhanced_max_speed,avg_speed,enhanced_avg_speed,max_cadence,avg_cadence,max_power,avg_power,max_heart_rate,avg_heart_rate,total_work
0,2020-05-22 16:16:16+00:00,2020-05-22 16:16:16+00:00,0.0,0.00,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0
1,2020-05-22 16:19:58+00:00,2020-05-22 16:16:16+00:00,222.0,1327.60,6.706,6.706,5.812,5.812,85,77,113,103,140,117,22866
2,2020-05-22 16:26:58+00:00,2020-05-22 16:19:58+00:00,420.0,3264.99,8.851,8.851,7.600,7.600,91,80,222,163,155,126,68460
3,2020-05-22 16:28:58+00:00,2020-05-22 16:26:58+00:00,120.0,849.25,8.851,8.851,7.153,7.153,93,87,166,111,157,139,13320
4,2020-05-22 16:30:58+00:00,2020-05-22 16:28:58+00:00,120.0,1014.07,8.673,8.673,8.494,8.494,85,53,213,208,148,125,24960
5,2020-05-22 16:31:58+00:00,2020-05-22 16:30:58+00:00,60.0,459.98,8.673,8.673,7.600,7.600,92,84,151,132,159,155,7920
6,2020-05-22 16:33:58+00:00,2020-05-22 16:31:58+00:00,120.0,1039.95,8.851,8.851,8.494,8.494,89,52,227,218,168,149,26160
7,2020-05-22 16:34:58+00:00,2020-05-22 16:33:58+00:00,60.0,464.23,8.851,8.851,7.600,7.600,91,85,218,133,168,163,7980
8,2020-05-22 16:36:58+00:00,2020-05-22 16:34:58+00:00,120.0,1059.12,9.030,9.030,8.941,8.941,86,55,236,230,166,157,27600
9,2020-05-22 16:37:58+00:00,2020-05-22 16:36:58+00:00,60.0,466.55,9.030,9.030,7.600,7.600,91,85,206,133,169,164,7980


In [25]:
pd.DataFrame(goat["session"]["data"], columns=dict(goat["session"]["schema"]).keys())

,timestamp,start_time,total_elapsed_time,total_timer_time,total_distance,training_stress_score,normalized_power,intensity_factor,total_moving_time,message_index,total_calories,sport,sub_sport,event,avg_speed,enhanced_avg_speed,max_speed,enhanced_max_speed,avg_cadence,max_cadence,avg_power,max_power,avg_heart_rate,max_heart_rate,threshold_power,total_work
0,2020-05-22 16:58:27+00:00,2020-05-22 16:16:16+00:00,2531.0,2531.0,19842.0,49.0,182,0.833,2531.0,0,417,cycling,indoor_cycling,activity,7.6,7.6,9.03,9.03,72,100,166,240,145,178,219,417632


In [26]:
pd.DataFrame(goat["activity"]["data"], columns=dict(goat["activity"]["schema"]).keys())

,timestamp,local_timestamp,num_sessions,type,event,event_type
0,2020-05-22 16:58:27+00:00,1989-12-31,1,manual,activity,start


In [61]:
def_record = parse_definition_record(fit_frames[1])
data_record = parse_data_message_record(fit_frames[56])

In [64]:
a = FitDataMessageRecord(
    is_developer_data=False,
    local_mesg_num=0,
    time_offset=None,
    def_mesg="parse_definition_record(record.def_mesg)",
    fields=[
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="timestamp",
                type=FitFieldType(
                    name="date_time",
                    base_type=FitBaseType(
                        name="uint32",
                        identifier=134,
                        fmt="I",
                        size=4,
                        parse="record.parse",
                    ),
                    enum="record.enum",
                ),
                def_num=253,
                scale=None,
                offset=None,
                units="s",
                components=None,
                subfields=None,
            ),
            parent_field=None,
            value=datetime.datetime(
                2020, 5, 22, 16, 16, 59, tzinfo=datetime.timezone.utc
            ),
            raw_value=959098619,
            units=None,
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="power",
                type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                def_num=7,
                scale=None,
                offset=None,
                units="watts",
                components=None,
                subfields=None,
            ),
            parent_field=None,
            value=110,
            raw_value=110,
            units="watts",
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="enhanced_speed",
                type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                def_num=73,
                scale=1000,
                offset=None,
                units="m/s",
                components=None,
                subfields=None,
            ),
            parent_field=None,
            value=5.812,
            raw_value=5.812,
            units="m/s",
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="speed",
                type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                def_num=6,
                scale=1000,
                offset=None,
                units="m/s",
                components=[
                    FitComponentField(
                        name="enhanced_speed",
                        def_num=73,
                        scale=1000,
                        offset=None,
                        units="m/s",
                        accumulate=False,
                        bits=16,
                        bit_offset=0,
                    )
                ],
                subfields=None,
            ),
            parent_field=None,
            value=5.812,
            raw_value=5812,
            units="m/s",
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="distance",
                type=FitBaseType(
                    name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
                ),
                def_num=5,
                scale=100,
                offset=None,
                units="m",
                components=None,
                subfields=None,
            ),
            parent_field=None,
            value=141.0,
            raw_value=14100,
            units="m",
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="cadence",
                type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                def_num=4,
                scale=None,
                offset=None,
                units="rpm",
                components=None,
                subfields=None,
            ),
            parent_field=None,
            value=72,
            raw_value=72,
            units="rpm",
        ),
        FitFieldData(
            field_def="parse_field_definition(record.field_def)",
            field=FitField(
                name="heart_rate",
                type=FitBaseType(
                    name="uint8", identifier=2, fmt="B", size=1, parse="record.parse"
                ),
                def_num=3,
                scale=None,
                offset=None,
                units="bpm",
                components=None,
                subfields=None,
            ),
            parent_field=None,
            value=112,
            raw_value=112,
            units="bpm",
        ),
    ],
    chunk=None,
)

In [57]:
FitDefinitionRow(
    is_developer_data=False,
    local_mesg_num=0,
    time_offset=None,
    mesg_type=FitMessageType(
        name="file_id",
        mesg_num=0,
        fields={
            0: FitField(
                name="type",
                type=FitFieldType(
                    name="file",
                    base_type=FitBaseType(
                        name="enum", identifier=0, fmt="B", size=1, parse="record.parse"
                    ),
                    enum="record.enum",
                ),
                def_num=0,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            1: FitField(
                name="manufacturer",
                type=FitFieldType(
                    name="manufacturer",
                    base_type=FitBaseType(
                        name="uint16",
                        identifier=132,
                        fmt="H",
                        size=2,
                        parse="record.parse",
                    ),
                    enum="record.enum",
                ),
                def_num=1,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            2: FitField(
                name="product",
                type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                def_num=2,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=[
                    FitSubField(
                        name="favero_product",
                        def_num=2,
                        type=FitFieldType(
                            name="favero_product",
                            base_type=FitBaseType(
                                name="uint16",
                                identifier=132,
                                fmt="H",
                                size=2,
                                parse="record.parse",
                            ),
                            enum="record.enum",
                        ),
                        scale=None,
                        offset=None,
                        units=None,
                        components=None,
                        ref_fields=[
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="favero_electronics",
                                raw_value=263,
                            )
                        ],
                    ),
                    FitSubField(
                        name="garmin_product",
                        def_num=2,
                        type=FitFieldType(
                            name="garmin_product",
                            base_type=FitBaseType(
                                name="uint16",
                                identifier=132,
                                fmt="H",
                                size=2,
                                parse="record.parse",
                            ),
                            enum="record.enum",
                        ),
                        scale=None,
                        offset=None,
                        units=None,
                        components=None,
                        ref_fields=[
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="garmin",
                                raw_value=1,
                            ),
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="dynastream",
                                raw_value=15,
                            ),
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="dynastream_oem",
                                raw_value=13,
                            ),
                        ],
                    ),
                ],
            ),
            3: FitField(
                name="serial_number",
                type=FitBaseType(
                    name="uint32z",
                    identifier=140,
                    fmt="I",
                    size=4,
                    parse="record.parse",
                ),
                def_num=3,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            4: FitField(
                name="time_created",
                type=FitFieldType(
                    name="date_time",
                    base_type=FitBaseType(
                        name="uint32",
                        identifier=134,
                        fmt="I",
                        size=4,
                        parse="record.parse",
                    ),
                    enum="record.enum",
                ),
                def_num=4,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            5: FitField(
                name="number",
                type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                def_num=5,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            8: FitField(
                name="product_name",
                type=FitBaseType(
                    name="string", identifier=7, fmt="s", size=1, parse="record.parse"
                ),
                def_num=8,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
        },
    ),
    global_mesg_num=0,
    endian="<",
    field_defs=[
        FitFieldDefinition(
            field=FitField(
                name="type",
                type=FitFieldType(
                    name="file",
                    base_type=FitBaseType(
                        name="enum", identifier=0, fmt="B", size=1, parse="record.parse"
                    ),
                    enum="record.enum",
                ),
                def_num=0,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            def_num=0,
            base_type=FitBaseType(
                name="enum", identifier=0, fmt="B", size=1, parse="record.parse"
            ),
            size=1,
        ),
        FitFieldDefinition(
            field=FitField(
                name="manufacturer",
                type=FitFieldType(
                    name="manufacturer",
                    base_type=FitBaseType(
                        name="uint16",
                        identifier=132,
                        fmt="H",
                        size=2,
                        parse="record.parse",
                    ),
                    enum="record.enum",
                ),
                def_num=1,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            def_num=1,
            base_type=FitBaseType(
                name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
            ),
            size=2,
        ),
        FitFieldDefinition(
            field=FitField(
                name="product",
                type=FitBaseType(
                    name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
                ),
                def_num=2,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=[
                    FitSubField(
                        name="favero_product",
                        def_num=2,
                        type=FitFieldType(
                            name="favero_product",
                            base_type=FitBaseType(
                                name="uint16",
                                identifier=132,
                                fmt="H",
                                size=2,
                                parse="record.parse",
                            ),
                            enum="record.enum",
                        ),
                        scale=None,
                        offset=None,
                        units=None,
                        components=None,
                        ref_fields=[
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="favero_electronics",
                                raw_value=263,
                            )
                        ],
                    ),
                    FitSubField(
                        name="garmin_product",
                        def_num=2,
                        type=FitFieldType(
                            name="garmin_product",
                            base_type=FitBaseType(
                                name="uint16",
                                identifier=132,
                                fmt="H",
                                size=2,
                                parse="record.parse",
                            ),
                            enum="record.enum",
                        ),
                        scale=None,
                        offset=None,
                        units=None,
                        components=None,
                        ref_fields=[
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="garmin",
                                raw_value=1,
                            ),
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="dynastream",
                                raw_value=15,
                            ),
                            FitReferenceField(
                                name="manufacturer",
                                def_num=1,
                                value="dynastream_oem",
                                raw_value=13,
                            ),
                        ],
                    ),
                ],
            ),
            def_num=2,
            base_type=FitBaseType(
                name="uint16", identifier=132, fmt="H", size=2, parse="record.parse"
            ),
            size=2,
        ),
        FitFieldDefinition(
            field=FitField(
                name="serial_number",
                type=FitBaseType(
                    name="uint32z",
                    identifier=140,
                    fmt="I",
                    size=4,
                    parse="record.parse",
                ),
                def_num=3,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            def_num=3,
            base_type=FitBaseType(
                name="uint32z", identifier=140, fmt="I", size=4, parse="record.parse"
            ),
            size=4,
        ),
        FitFieldDefinition(
            field=FitField(
                name="time_created",
                type=FitFieldType(
                    name="date_time",
                    base_type=FitBaseType(
                        name="uint32",
                        identifier=134,
                        fmt="I",
                        size=4,
                        parse="record.parse",
                    ),
                    enum="record.enum",
                ),
                def_num=4,
                scale=None,
                offset=None,
                units=None,
                components=None,
                subfields=None,
            ),
            def_num=4,
            base_type=FitBaseType(
                name="uint32", identifier=134, fmt="I", size=4, parse="record.parse"
            ),
            size=4,
        ),
    ],
    dev_field_defs=[],
    chunk=None,
)

FitDefinitionRow(is_developer_data=False, local_mesg_num=0, time_offset=None, mesg_type=FitMessageType(name='file_id', mesg_num=0, fields={0: FitField(name='type', type=FitFieldType(name='file', base_type=FitBaseType(name='enum', identifier=0, fmt='B', size=1, parse='record.parse'), enum='record.enum'), def_num=0, scale=None, offset=None, units=None, components=None, subfields=None), 1: FitField(name='manufacturer', type=FitFieldType(name='manufacturer', base_type=FitBaseType(name='uint16', identifier=132, fmt='H', size=2, parse='record.parse'), enum='record.enum'), def_num=1, scale=None, offset=None, units=None, components=None, subfields=None), 2: FitField(name='product', type=FitBaseType(name='uint16', identifier=132, fmt='H', size=2, parse='record.parse'), def_num=2, scale=None, offset=None, units=None, components=None, subfields=[FitSubField(name='favero_product', def_num=2, type=FitFieldType(name='favero_product', base_type=FitBaseType(name='uint16', identifier=132, fmt='H', size

In [14]:
pd.DataFrame(
    fit_classes["record"]["data"],
    columns=[col for col, typ in fit_classes["record"]["schema"]],
)

,timestamp,power,speed,enhanced_speed,distance,cadence,heart_rate
0,2020-05-22 16:16:16+00:00,0,0.000,0.000,0.0,0,0
1,2020-05-22 16:16:16+00:00,0,0.000,0.000,0.0,0,0
2,2020-05-22 16:16:16+00:00,0,0.000,0.000,0.0,2,133
3,2020-05-22 16:16:17+00:00,0,0.000,0.000,0.0,2,133
4,2020-05-22 16:16:18+00:00,22,0.000,0.000,0.0,2,134
...,...,...,...,...,...,...,...
95,2020-05-22 16:17:47+00:00,110,6.661,6.661,451.0,83,106
96,2020-05-22 16:17:48+00:00,110,6.661,6.661,458.0,84,108
97,2020-05-22 16:17:49+00:00,110,6.661,6.661,464.0,84,112
98,2020-05-22 16:17:50+00:00,110,6.661,6.661,471.0,84,112


In [121]:
[key for key, typ in fit_classes["record"]["schema"]]

['timestamp',
 'power',
 'speed',
 'enhanced_speed',
 'distance',
 'cadence',
 'heart_rate']

In [130]:
key = "activity"
pd.DataFrame(
    fit_classes[key]["data"], columns=[key for key, typ in fit_classes[key]["schema"]],
).head(50)

,timestamp,local_timestamp,num_sessions,type,event,event_type
0,2020-05-22 16:58:27+00:00,1989-12-31,1,manual,activity,start
